<h3 style='opacity: 0.5'> To Run a basic example on the Julia top-level: </h3>

In [ ]:
using GeneralizedChartparsing
grammar = make_jazz_grammar();
train!(grammar, make_parsable_trees());
sequence = split("C^7 Dm7 G7 C^7");
forest = run_chartparser(sequence, grammar);
best_tree(forest)

<pre>

The Grammar type:
    The Grammar type contains a start State type. The state is continually updated and added to. 
    A typical CFG type Grammar has a startstate, a list of accepted startsymbols, a list of non-terminal
    ContextFreeRules (category rule type), a list of terminal ContextFreeRules (terminal rule type), 
    and a score.

The State type:
    The State type contains tuples of completed categories and rules along with a dictionary of possible
    transitions from these rules. The state represents completed rules in the syntax tree and possible 
    transitions from these rules to new future states. Using these strucutres, the parser can pursue 
    different parses by checking allowed transitions from different category type.   
</pre>

<pre>
NOTE: these types are specific to the Grammar type and are parametrized as so in the initialize() function 
    T Terminal Type : string argument i.e. "eats" 
    C Category Type: basic type i.e. V, N... 
    CR Category Rule Type : non-terminal clause, i.e. VP -> V NP 
    TR Terminal Rule Type : terminal clause, i.e. V -> "eats" 
    S Score
</pre>

In [ ]:
type Grammar{C, T, CR, TR, S, Cond}
    startstate    :: State{C, CR}
    startsymbols  :: Vector{C}
    terminal_dict :: Dict{T, Vector{Tuple{C, TR}}}
    rule_cond     :: Cond
    prior_cond    :: Cond # needed for GibbsSampling
end

type State{C, CR} # category and rule
    comp    :: Vector{Tuple{C, CR}}  # completions: tuples of categories and rules
    trans   :: Dict{C, State{C, CR}} # possible transitions
    isfinal :: Bool
end


<pre>
   The run_chartparser(sequence, grammar) function will initialize a chart, an agenda, and a logbook. 
   
   The Chart class: 
       The chart is just a list of chart cells. 
       
   The ChartCell class:
       Each chart cell contains a dictionary of constituents currently at that cell and a dictionary 
       of states which are instantiated with unit or non-unit clauses from the program during parsing.
   
   The Agenda class:
       The agenda is just a priority queue used to pursue different parses. The parser needs a ranking 
       in order to pursue a certain parse all the way through. 
   
   The ParserLogbook class:
       The parser logbook is essentially a replication of the chart, however, it contains additional information
       such as a list of edge states which have been currently instantiated and a list of constituents currently
       in use. 
       

</pre>

In [ ]:
type ChartCell{C,St}
    edgeids :: Dict{St, Vector{Int}}
    consids :: Dict{C, Vector{Int}}
end

type Chart{C,St}
    cells :: Vector{ChartCell{C,St}}
end

mutable struct Agenda{T}
    pqueue :: PriorityQueue{Int, T}
end

type ParserLogbook{C,T,CR,TR,St,S}
    edges :: Vector{Edge{St,S}}
    conss :: Vector{Constituent{C,T,CR,TR,S}}
    edgeids :: Dict{EdgeKey{St}, Int}
    consids :: Dict{ConsKey{C}, Int}
end

<pre>

The follwing function (run_chartparser) runs the entirety of the parse algorithm. The method will first 
instantiate a chart, agenda, and logbook as described above. The function will run until the agenda
(priority queue) is empty. 

At each iteration of the algorithm, the id with lowest priority is dequeued from the agenda. 
    Ids with positive values are Edges
    Ids with negative values are Constituents

Edges:
    Edges represent edges in a standard syntax tree. They are the vessel for transitioning between rules 
    and relate constituents together. Edges are used to evaluate possible traversals (paths) throughout the 
    tree.    

Constituents:
    Constituents represent constituents in a grammar. They hold information about paths (EdgeCompletions)
    from parent nodes in the tree. Constituents are used to represent the structure of the tree. 


</pre>

<pre>

There are two primary options during execution:
    1) The current dequeued id from the agenda is an Edge
    2) The current dequeued id from the agenda is a Constituent 

1) The current dequeued id from the agenda is an Edge:
    finish!(get_edge(logbook, id), chart, agenda, logbook, grammar)::Void



2) The current dequeued id from the agenda is a Constituent:
    Case a: 
        It is the initial phase of the program and the constituent is not on the "inside" of the tree.
        If the grammar allows a transition from the startstate of the grammar to the current category
        of the constituent, then the state is transitioned from the grammar startstate to the constituent
        category and EdgeKey is created which spans the constituent in the string and contains the state
        transition. A traversal is created which keeps track of the new constituent id and the score for
        that new constituent. Finally, the EdgeKey and Traversal are added to the agenda and logbook. 
        
    Case b:  
        It is not the initial phase of the program, the constituent is an internal node,
        AND the chart does not contain the current constituent. 
            Then insert the category of the constituent and its corresponding id into the correct chart cell. 
            As stated earlier, ChartCells contain dictionaries of edge ids and constituent ids. 
            Constituent keys and values are categories in a grammar and unique numeric ids respectively.
            Edge keys and values are states and unique numeric ids. The keys in the edgeid dictionary at
            each chart cell contains all the information needed at that point in the tree. As stated earlier,
            States contain many tuples of categories and rules (completed rules so far) and a possible
            transition dictionary. Keys are possible categories for the next level in the tree along with
            possible rules which can be applied at that moment. The State type in itself is recursive
            (its transition values are themselves States). This essentially allows the program to look ahead
            in the parse in order to determine possible transitions.
            
            The follwoing function is then called:
            
                do_fundamental_rule!(edge, chart, agenda, logbook, grammar) 
            
            This function will evaluate each possible transition state (edgeids(chart, cons) returns
            a dictionary of possible state transitions from the constituent at a certain chart cell) 
            from the current constituent. If the grammar allows a transition from the category of the
            current constituent to the next state in question, each edge involved in this transition
            is further evaluated. get_edge(logbook, edgeid) is called which will return a positive value
            representing an edge. The next possible state for the given category is then retrieved
            (transition(grammar, state, cat(cons)) returns the next possible transition for the given 
            category of the constituent in question. An EdgeKey is created which represents the path
            from the current constituent to the next possible state. A Traversal is created which 
            stores information about the current transition (the constituent and path (edge) that
            the parser is taking). The Traversal type is used to keep track of transitions between 
            edges and constituents in a tree and also compare different transitions in a tree. Traversals
            contain a score or cost ( score(edge) * score(cons) ) for taking a certain path in the tree. 
            The agenda and logbook are then updated with the new EdgeKey and the new Traversal. 
            Therefore, each cell contains all possible transitions from that current category to the next state. 
            
            Note that this process is similar to what occurs at the start of the parse. Instead of
            transitioning from the grammar startstate to a category, we are evaluating different posssible
            transitions from the current category to different future states. 
            
        
       
        
</pre>

In [ ]:
function run_chartparser(input::Vector, grammar, dependency_matrix::AbstractMatrix{Bool}, parsing_method=:full; epsilon=missing)
    input = map(terminal_type(grammar), input) :: Vector{terminal_type(grammar)}
    chart, agenda, logbook = initialize(input, grammar, parsing_method, epsilon)
    while !isempty(agenda)
        # finish = do inference and insert or accumulate
        id = dequeue!(agenda)
        if id > 0 # then id is an edge id
            finish!(get_edge(logbook, id), chart, agenda, logbook, grammar)::Void
        else
            cons = get_cons(logbook, id)
            # println(dependency_matrix[start(cons), tail(cons)])
            if dependency_matrix[start(cons), tail(cons)]
                # println(cons)
                finish!(cons, chart, agenda, logbook, grammar)::Void

                if parsing_method == :viterbi && length(cons) == length(input) && cat(cons) in startsymbols(grammar)
                    break
                end
            end
        end
    end
    ParseForest(chart, logbook, input, grammar)
end

In [ ]:

struct ConsKey{C}
    start :: Int
    tail  :: Int
    cat   :: C
end

struct EdgeKey{St}
    start :: Int
    tail  :: Int
    state :: St
end

struct EdgeCompletion{CR,S} <: Completion{S}
    edgeid :: Int
    rule   :: CR
    score  :: S
    inloop :: Bool
end

struct Traversal{S}
    edgeid  :: Int
    consid  :: Int
    score   :: S
    inloop  :: Bool
end

type Edge{St,S} <: Item
    start              :: Int
    tail               :: Int
    state              :: St
    score              :: S
    isfinished         :: Bool
    traversals         :: Vector{Traversal{S}}
    id                 :: Int # edges have positive ids
    lastpopscore       :: S
    lastoutsidepopscore:: S
    insidepopnumber    :: Int
    outsidepopnumber   :: Int
    outside_finished   :: Bool
    trav_outs_summands :: Dict{Tuple{Int, Int}, LogProb}
    comp_outs_summands :: Dict{Int, LogProb}
end

type Constituent{C,T,CR,TR,S} <: Item
    start               :: Int
    tail                :: Int
    cat                 :: C
    score               :: S
    isfinished          :: Bool
    completions         :: Vector{EdgeCompletion{CR,S}}
    terminal_completion :: Nullable{TerminalCompletion{T,TR,S}}
    id                  :: Int # constituents have negative ids
    lastpopscore        :: S
    lastoutsidepopscore :: S
    insidepopnumber     :: Int
    outsidepopnumber    :: Int
    outside_finished    :: Bool
    trav_outs_summands  :: Dict{Tuple{Int, Int}, LogProb}
end


